In [1]:
import pandas as pd
import numpy as np

DATA_PATH = "data/processed"

df = pd.read_parquet(f"{DATA_PATH}/monthly_full.parquet")

print(df.shape)
df.head()


(10913850, 5)


,date_block_num,shop_id,item_id,item_cnt_month,item_cnt_month_clipped
0,0,59,22154,1.0,1.0
1,0,59,2552,0.0,0.0
2,0,59,2554,0.0,0.0
3,0,59,2555,0.0,0.0
4,0,59,2564,0.0,0.0


In [2]:
df = df.sort_values(["date_block_num", "shop_id", "item_id"]).reset_index(drop=True)

df[["date_block_num", "shop_id", "item_id"]].head()


,date_block_num,shop_id,item_id
0,0,0,19
1,0,0,27
2,0,0,28
3,0,0,29
4,0,0,32


In [3]:
def add_lag(df, lags, col):
    for lag in lags:
        df[f"{col}_lag_{lag}"] = (
            df
            .groupby(["shop_id", "item_id"])[col]
            .shift(lag)
        )
    return df


In [4]:
LAGS = [1, 2, 3, 6, 12]

df = add_lag(df, LAGS, "item_cnt_month_clipped")

df[[f"item_cnt_month_clipped_lag_{l}" for l in LAGS]].head(15)


,item_cnt_month_clipped_lag_1,item_cnt_month_clipped_lag_2,item_cnt_month_clipped_lag_3,item_cnt_month_clipped_lag_6,item_cnt_month_clipped_lag_12
0,NaN,NaN,NaN,NaN,NaN
1,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN
5,NaN,NaN,NaN,NaN,NaN
6,NaN,NaN,NaN,NaN,NaN
7,NaN,NaN,NaN,NaN,NaN
8,NaN,NaN,NaN,NaN,NaN
9,NaN,NaN,NaN,NaN,NaN


In [5]:
lag_cols = [f"item_cnt_month_clipped_lag_{l}" for l in LAGS]

df[lag_cols].isna().mean().sort_values(ascending=False)


item_cnt_month_clipped_lag_12    0.745040
item_cnt_month_clipped_lag_6     0.469699
item_cnt_month_clipped_lag_3     0.269581
item_cnt_month_clipped_lag_2     0.191327
item_cnt_month_clipped_lag_1     0.104107
dtype: float64

In [6]:
df[lag_cols] = df[lag_cols].fillna(0)

df[lag_cols].describe()


,item_cnt_month_clipped_lag_1,item_cnt_month_clipped_lag_2,item_cnt_month_clipped_lag_3,item_cnt_month_clipped_lag_6,item_cnt_month_clipped_lag_12
count,1.091385e+07,1.091385e+07,1.091385e+07,1.091385e+07,1.091385e+07
mean,2.882394e-01,2.768729e-01,2.651453e-01,2.282167e-01,1.413480e-01
std,1.205493e+00,1.185778e+00,1.167193e+00,1.104924e+00,9.053981e-01
min,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
25%,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
50%,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
75%,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
max,2.000000e+01,2.000000e+01,2.000000e+01,2.000000e+01,2.000000e+01


In [7]:
# Promedio mensual de ventas por tienda 
shop_month_mean = (
    df.groupby(["date_block_num", "shop_id"], as_index=False)["item_cnt_month_clipped"]
      .mean()
      .rename(columns={"item_cnt_month_clipped": "shop_month_mean"})
)

# Promedio mensual de ventas por producto 
item_month_mean = (
    df.groupby(["date_block_num", "item_id"], as_index=False)["item_cnt_month_clipped"]
      .mean()
      .rename(columns={"item_cnt_month_clipped": "item_month_mean"})
)

df = df.merge(shop_month_mean, on=["date_block_num", "shop_id"], how="left")
df = df.merge(item_month_mean, on=["date_block_num", "item_id"], how="left")

df[["shop_month_mean", "item_month_mean"]].describe()



,shop_month_mean,item_month_mean
count,1.091385e+07,1.091385e+07
mean,2.982399e-01,2.982399e-01
std,2.197987e-01,8.483580e-01
min,0.000000e+00,0.000000e+00
25%,1.722945e-01,3.921569e-02
50%,2.341354e-01,8.695652e-02
75%,3.216047e-01,2.391304e-01
max,1.799243e+00,1.955556e+01


In [8]:
for col in ["shop_month_mean", "item_month_mean"]:
    df = add_lag(df, [1], col)

df[["shop_month_mean_lag_1", "item_month_mean_lag_1"]].isna().mean()


shop_month_mean_lag_1    0.104107
item_month_mean_lag_1    0.104107
dtype: float64

In [9]:
df[["shop_month_mean_lag_1", "item_month_mean_lag_1"]] = df[["shop_month_mean_lag_1", "item_month_mean_lag_1"]].fillna(0)
df[["shop_month_mean_lag_1", "item_month_mean_lag_1"]].describe()


,shop_month_mean_lag_1,item_month_mean_lag_1
count,1.091385e+07,1.091385e+07
mean,2.694319e-01,2.871383e-01
std,2.277702e-01,8.403510e-01
min,0.000000e+00,0.000000e+00
25%,1.492471e-01,2.222222e-02
50%,2.231922e-01,8.333333e-02
75%,3.085226e-01,2.272727e-01
max,1.799243e+00,1.955556e+01


In [10]:
df.isna().mean().sort_values(ascending=False).head(15)


date_block_num                   0.0
shop_id                          0.0
item_id                          0.0
item_cnt_month                   0.0
item_cnt_month_clipped           0.0
item_cnt_month_clipped_lag_1     0.0
item_cnt_month_clipped_lag_2     0.0
item_cnt_month_clipped_lag_3     0.0
item_cnt_month_clipped_lag_6     0.0
item_cnt_month_clipped_lag_12    0.0
shop_month_mean                  0.0
item_month_mean                  0.0
shop_month_mean_lag_1            0.0
item_month_mean_lag_1            0.0
dtype: float64

In [11]:
OUT_PATH = "data/processed/features.parquet"

df.to_parquet(OUT_PATH, index=False)

print("Features guardadas en:", OUT_PATH)
print("Shape final:", df.shape)



Features guardadas en: data/processed/features.parquet
Shape final: (10913850, 14)
